# Numerical Integration - Multiple Variables
- **Purpose**: Compute double, triple, and n-dimensional integrals
- **scipy.integrate**: dblquad, tplquad, nquad for multi-dimensional integration
- **Applications**: Volume, surface area, center of mass, joint probabilities, physics

Key functions:
- **dblquad()**: Double integrals (2D)
- **tplquad()**: Triple integrals (3D)
- **nquad()**: N-dimensional integrals

Real examples:
- Volume under surfaces
- Joint probability distributions
- Moment of inertia and center of mass
- Spherical and cylindrical coordinates

In [1]:
import numpy as np
from scipy import integrate
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Set print options
np.set_printoptions(precision=6, suppress=True)

print("Multi-dimensional integration module loaded")

Multi-dimensional integration module loaded


## Double Integration

Compute integral over 2D region:

\[ I = \int_{y=c}^{y=d} \int_{x=a}^{x=b} f(x, y) \, dx \, dy \]

Or with variable limits:

\[ I = \int_{y=c}^{y=d} \int_{x=g_1(y)}^{x=g_2(y)} f(x, y) \, dx \, dy \]

**Interpretation**:
- Volume under surface z = f(x,y)
- Total mass of 2D density distribution
- Probability over 2D region

**Order matters**: Inner integral first, outer integral second

## dblquad() - Double Integration

**Function**: 
```python
scipy.integrate.dblquad(func, a, b, gfun, hfun)
```

**Parameters**:
- `func(y, x)`: Function to integrate (note: **y first, x second**)
- `a, b`: Fixed x limits
- `gfun(x)`: Lower y limit (function of x)
- `hfun(x)`: Upper y limit (function of x)

**Returns**: `(result, error_estimate)`

**Integration order**: ∫ₐᵇ ∫_{gfun(x)}^{hfun(x)} f(x,y) dy dx

**Important**: First argument to func is y, second is x!

In [2]:
# Example 1: ∫₀¹ ∫₀¹ xy dx dy = ∫₀¹ [x²y/2]₀¹ dy = ∫₀¹ y/2 dy = 1/4

def f1(y, x):  # Note: y comes first!
    return x * y

# Constant limits: x from 0 to 1, y from 0 to 1
result, error = integrate.dblquad(
    f1,
    0, 1,          # x limits
    lambda x: 0,   # y lower limit (function of x)
    lambda x: 1    # y upper limit (function of x)
)

print("Example 1: ∫₀¹ ∫₀¹ xy dy dx")
print(f"  Result: {result:.10f}")
print(f"  Expected: 0.25")
print(f"  Error: {error:.2e}")

# Example 2: ∫₀² ∫₀³ (x² + y²) dx dy
def f2(y, x):
    return x**2 + y**2

result2, error2 = integrate.dblquad(
    f2,
    0, 2,
    lambda x: 0,
    lambda x: 3
)

print("\nExample 2: ∫₀² ∫₀³ (x² + y²) dy dx")
print(f"  Result: {result2:.10f}")
# Analytical: ∫₀² ∫₀³ (x² + y²) dy dx = ∫₀² [x²y + y³/3]₀³ dx
#           = ∫₀² (3x² + 9) dx = [x³ + 9x]₀² = 8 + 18 = 26
print(f"  Expected: 26.0")
print(f"  Error: {error2:.2e}")

Example 1: ∫₀¹ ∫₀¹ xy dy dx
  Result: 0.2500000000
  Expected: 0.25
  Error: 5.54e-15

Example 2: ∫₀² ∫₀³ (x² + y²) dy dx
  Result: 26.0000000000
  Expected: 26.0
  Error: 2.89e-13


## Variable Limits: Integrating Over Non-Rectangular Regions

**Example**: Triangle with vertices (0,0), (1,0), (1,1)

Region defined by:
- 0 ≤ x ≤ 1
- 0 ≤ y ≤ x

\[ \int_0^1 \int_0^x f(x,y) \, dy \, dx \]

The limits gfun(x) and hfun(x) define the y-range for each x

In [3]:
# Area of triangle: ∫₀¹ ∫₀ˣ 1 dy dx = ∫₀¹ x dx = 1/2

def constant_one(y, x):
    return 1.0

# x from 0 to 1, y from 0 to x
area, error = integrate.dblquad(
    constant_one,
    0, 1,           # x limits
    lambda x: 0,    # y lower = 0
    lambda x: x     # y upper = x (triangle)
)

print("Triangle area (vertices: (0,0), (1,0), (1,1))")
print(f"  Computed area: {area:.10f}")
print(f"  Expected (base×height/2): {0.5 * 1 * 1:.10f}")
print(f"  Error: {error:.2e}")

# Volume under plane z = x + y over triangle
def plane(y, x):
    return x + y

volume, error = integrate.dblquad(
    plane,
    0, 1,
    lambda x: 0,
    lambda x: x
)

print("\nVolume under z = x+y over triangle:")
print(f"  Computed volume: {volume:.10f}")
# Analytical: ∫₀¹ ∫₀ˣ (x+y) dy dx = ∫₀¹ [xy + y²/2]₀ˣ dx
#           = ∫₀¹ (x² + x²/2) dx = ∫₀¹ 3x²/2 dx = x³/2|₀¹ = 1/2
print(f"  Expected: {0.5:.10f}")
print(f"  Error: {error:.2e}")

Triangle area (vertices: (0,0), (1,0), (1,1))
  Computed area: 0.5000000000
  Expected (base×height/2): 0.5000000000
  Error: 1.11e-14

Volume under z = x+y over triangle:
  Computed volume: 0.5000000000
  Expected: 0.5000000000
  Error: 1.66e-14


## Example: Circular Region

Integrate over disk: x² + y² ≤ r²

For fixed x, y ranges from -√(r²-x²) to √(r²-x²)

\[ \int_{-r}^{r} \int_{-\sqrt{r^2-x^2}}^{\sqrt{r^2-x^2}} f(x,y) \, dy \, dx \]

**Application**: Area of circle = πr²

In [4]:
# Area of circle with radius 2
r = 2.0

def circle_lower(x):
    return -np.sqrt(r**2 - x**2)

def circle_upper(x):
    return np.sqrt(r**2 - x**2)

area, error = integrate.dblquad(
    lambda y, x: 1.0,
    -r, r,
    circle_lower,
    circle_upper
)

print(f"Area of circle with radius {r}:")
print(f"  Computed: {area:.10f}")
print(f"  Expected (πr²): {np.pi * r**2:.10f}")
print(f"  Error: {error:.2e}")

# Volume of hemisphere z = √(r² - x² - y²) over circular base
def hemisphere(y, x):
    return np.sqrt(r**2 - x**2 - y**2)

volume, error = integrate.dblquad(
    hemisphere,
    -r, r,
    circle_lower,
    circle_upper
)

print(f"\nVolume of hemisphere (radius {r}):")
print(f"  Computed: {volume:.10f}")
print(f"  Expected (2πr³/3): {2*np.pi*r**3/3:.10f}")
print(f"  Relative error: {abs(volume - 2*np.pi*r**3/3)/(2*np.pi*r**3/3)*100:.4f}%")

Area of circle with radius 2.0:
  Computed: 12.5663706144
  Expected (πr²): 12.5663706144
  Error: 8.00e-09

Volume of hemisphere (radius 2.0):
  Computed: 16.7551608191
  Expected (2πr³/3): 16.7551608191
  Relative error: 0.0000%


## Real Example: Joint Probability Distribution

**Bivariate Normal Distribution**:

\[ f(x,y) = \frac{1}{2\pi\sigma_x\sigma_y\sqrt{1-\rho^2}} \exp\left(-\frac{1}{2(1-\rho^2)}\left[\frac{x^2}{\sigma_x^2} + \frac{y^2}{\sigma_y^2} - \frac{2\rho xy}{\sigma_x\sigma_y}\right]\right) \]

Where:
- σₓ, σᵧ: Standard deviations
- ρ: Correlation coefficient

**Application**: Find P(X < 1, Y < 1) for correlated random variables

In [5]:
# Bivariate normal with correlation
def bivariate_normal(y, x, sigma_x=1.0, sigma_y=1.0, rho=0.5):
    """Bivariate normal PDF (mean = 0)"""
    z = (x**2/sigma_x**2 + y**2/sigma_y**2 - 
         2*rho*x*y/(sigma_x*sigma_y)) / (1 - rho**2)
    coeff = 1 / (2*np.pi*sigma_x*sigma_y*np.sqrt(1-rho**2))
    return coeff * np.exp(-z/2)

# Verify normalization: ∫∫ f(x,y) dx dy = 1
total_prob, _ = integrate.dblquad(
    lambda y, x: bivariate_normal(y, x, rho=0.5),
    -np.inf, np.inf,
    lambda x: -np.inf,
    lambda x: np.inf
)

print("Bivariate Normal Distribution (σₓ=1, σᵧ=1, ρ=0.5)")
print(f"  Total probability: {total_prob:.10f}")
print(f"  (Should be 1.0)")

# P(X < 1, Y < 1)
prob, _ = integrate.dblquad(
    lambda y, x: bivariate_normal(y, x, rho=0.5),
    -np.inf, 1,
    lambda x: -np.inf,
    lambda x: 1
)

print(f"\nP(X < 1, Y < 1): {prob:.6f}")

# Compare with independent case (ρ = 0)
prob_indep, _ = integrate.dblquad(
    lambda y, x: bivariate_normal(y, x, rho=0.0),
    -np.inf, 1,
    lambda x: -np.inf,
    lambda x: 1
)

print(f"P(X < 1, Y < 1) [independent, ρ=0]: {prob_indep:.6f}")
print(f"\nCorrelation increases joint probability!")
print(f"Increase: {(prob/prob_indep - 1)*100:.2f}%")

Bivariate Normal Distribution (σₓ=1, σᵧ=1, ρ=0.5)
  Total probability: 1.0000000000
  (Should be 1.0)

P(X < 1, Y < 1): 0.745204
P(X < 1, Y < 1) [independent, ρ=0]: 0.707861

Correlation increases joint probability!
Increase: 5.28%


## Physics Example: Center of Mass

For 2D region R with density ρ(x,y):

**Total mass**:
\[ M = \iint_R \rho(x,y) \, dA \]

**Center of mass coordinates**:
\[ \bar{x} = \frac{1}{M} \iint_R x\rho(x,y) \, dA \]
\[ \bar{y} = \frac{1}{M} \iint_R y\rho(x,y) \, dA \]

**Example**: Triangular plate with variable density ρ(x,y) = x + y

In [6]:
# Triangular region: 0 ≤ x ≤ 1, 0 ≤ y ≤ x
# Density: ρ(x,y) = x + y

def density(y, x):
    return x + y

# Total mass M = ∫∫ ρ dA
mass, _ = integrate.dblquad(
    density,
    0, 1,
    lambda x: 0,
    lambda x: x
)

print("Triangular plate with density ρ(x,y) = x + y")
print(f"  Total mass M: {mass:.10f}")

# x-coordinate of center of mass: x̄ = (1/M) ∫∫ x·ρ dA
moment_x, _ = integrate.dblquad(
    lambda y, x: x * density(y, x),
    0, 1,
    lambda x: 0,
    lambda x: x
)
x_bar = moment_x / mass

# y-coordinate of center of mass: ȳ = (1/M) ∫∫ y·ρ dA
moment_y, _ = integrate.dblquad(
    lambda y, x: y * density(y, x),
    0, 1,
    lambda x: 0,
    lambda x: x
)
y_bar = moment_y / mass

print(f"\nCenter of mass: ({x_bar:.6f}, {y_bar:.6f})")

# For uniform density (ρ = 1), centroid is at (2/3, 1/3)
mass_uniform, _ = integrate.dblquad(
    lambda y, x: 1.0,
    0, 1,
    lambda x: 0,
    lambda x: x
)
x_uniform, _ = integrate.dblquad(
    lambda y, x: x,
    0, 1,
    lambda x: 0,
    lambda x: x
)
y_uniform, _ = integrate.dblquad(
    lambda y, x: y,
    0, 1,
    lambda x: 0,
    lambda x: x
)

print(f"\nUniform density centroid: ({x_uniform/mass_uniform:.6f}, {y_uniform/mass_uniform:.6f})")
print(f"Expected: (0.666667, 0.333333)")
print(f"\nVariable density shifts center of mass!")

Triangular plate with density ρ(x,y) = x + y
  Total mass M: 0.5000000000

Center of mass: (0.750000, 0.416667)

Uniform density centroid: (0.666667, 0.333333)
Expected: (0.666667, 0.333333)

Variable density shifts center of mass!


## Triple Integration

Integrate over 3D regions:

\[ I = \int \int \int f(x,y,z) \, dz \, dy \, dx \]

**Function**: 
```python
scipy.integrate.tplquad(func, a, b, gfun, hfun, qfun, rfun)
```

**Parameters**:
- `func(z, y, x)`: Function (note order: **z, y, x**)
- `a, b`: x limits
- `gfun(x), hfun(x)`: y limits (functions of x)
- `qfun(x,y), rfun(x,y)`: z limits (functions of x, y)

**Applications**: Volume, mass, moment of inertia

In [7]:
# Example 1: Volume of rectangular box
# ∫₀¹ ∫₀² ∫₀³ 1 dz dy dx = 1×2×3 = 6

volume, error = integrate.tplquad(
    lambda z, y, x: 1.0,  # Note: z, y, x order!
    0, 1,                  # x limits
    lambda x: 0,           # y lower
    lambda x: 2,           # y upper
    lambda x, y: 0,        # z lower
    lambda x, y: 3         # z upper
)

print("Volume of box [0,1] × [0,2] × [0,3]:")
print(f"  Computed: {volume:.10f}")
print(f"  Expected: {1*2*3:.10f}")
print(f"  Error: {error:.2e}")

# Example 2: ∫₀¹ ∫₀¹ ∫₀¹ xyz dz dy dx = 1/8
result, error = integrate.tplquad(
    lambda z, y, x: x*y*z,
    0, 1,
    lambda x: 0,
    lambda x: 1,
    lambda x, y: 0,
    lambda x, y: 1
)

print("\n∫₀¹ ∫₀¹ ∫₀¹ xyz dz dy dx:")
print(f"  Computed: {result:.10f}")
print(f"  Expected: {1/8:.10f}")
print(f"  Error: {error:.2e}")

Volume of box [0,1] × [0,2] × [0,3]:
  Computed: 6.0000000000
  Expected: 6.0000000000
  Error: 6.66e-14

∫₀¹ ∫₀¹ ∫₀¹ xyz dz dy dx:
  Computed: 0.1250000000
  Expected: 0.1250000000
  Error: 5.53e-15


## Example: Volume of Sphere

Sphere: x² + y² + z² ≤ r²

For fixed x, y: -√(r²-x²-y²) ≤ z ≤ √(r²-x²-y²)

For fixed x: -√(r²-x²) ≤ y ≤ √(r²-x²)

\[ V = \int_{-r}^r \int_{-\sqrt{r^2-x^2}}^{\sqrt{r^2-x^2}} \int_{-\sqrt{r^2-x^2-y^2}}^{\sqrt{r^2-x^2-y^2}} 1 \, dz \, dy \, dx \]

Expected: V = (4/3)πr³

In [8]:
# Volume of sphere with radius 1
r = 1.0

def z_lower(x, y):
    return -np.sqrt(r**2 - x**2 - y**2)

def z_upper(x, y):
    return np.sqrt(r**2 - x**2 - y**2)

def y_lower(x):
    return -np.sqrt(r**2 - x**2)

def y_upper(x):
    return np.sqrt(r**2 - x**2)

volume, error = integrate.tplquad(
    lambda z, y, x: 1.0,
    -r, r,      # x limits
    y_lower, y_upper,  # y limits
    z_lower, z_upper   # z limits
)

expected_volume = 4*np.pi*r**3/3

print(f"Volume of sphere (radius {r}):")
print(f"  Computed: {volume:.10f}")
print(f"  Expected (4πr³/3): {expected_volume:.10f}")
print(f"  Relative error: {abs(volume - expected_volume)/expected_volume * 100:.4f}%")
print(f"\nNote: This is computationally expensive!")
print(f"      Spherical coordinates would be more efficient.")

Volume of sphere (radius 1.0):
  Computed: 4.1887902048
  Expected (4πr³/3): 4.1887902048
  Relative error: 0.0000%

Note: This is computationally expensive!
      Spherical coordinates would be more efficient.


## Example: Tetrahedron Volume

**Tetrahedron** with vertices at (0,0,0), (1,0,0), (0,1,0), (0,0,1)

Region defined by:
- 0 ≤ x ≤ 1
- 0 ≤ y ≤ 1-x
- 0 ≤ z ≤ 1-x-y

The plane x + y + z = 1 forms the slanted face.

Expected volume: 1/6

In [9]:
# Volume of tetrahedron
volume, error = integrate.tplquad(
    lambda z, y, x: 1.0,
    0, 1,                        # x: 0 to 1
    lambda x: 0,                 # y: 0 to 1-x
    lambda x: 1-x,
    lambda x, y: 0,              # z: 0 to 1-x-y
    lambda x, y: 1-x-y
)

print("Tetrahedron with vertices (0,0,0), (1,0,0), (0,1,0), (0,0,1)")
print(f"  Computed volume: {volume:.10f}")
print(f"  Expected (1/6): {1/6:.10f}")
print(f"  Error: {error:.2e}")

# Center of mass (uniform density)
# By symmetry, should be at (1/4, 1/4, 1/4)
x_moment, _ = integrate.tplquad(
    lambda z, y, x: x,
    0, 1,
    lambda x: 0, lambda x: 1-x,
    lambda x, y: 0, lambda x, y: 1-x-y
)

y_moment, _ = integrate.tplquad(
    lambda z, y, x: y,
    0, 1,
    lambda x: 0, lambda x: 1-x,
    lambda x, y: 0, lambda x, y: 1-x-y
)

z_moment, _ = integrate.tplquad(
    lambda z, y, x: z,
    0, 1,
    lambda x: 0, lambda x: 1-x,
    lambda x, y: 0, lambda x, y: 1-x-y
)

x_cm = x_moment / volume
y_cm = y_moment / volume
z_cm = z_moment / volume

print(f"\nCenter of mass: ({x_cm:.6f}, {y_cm:.6f}, {z_cm:.6f})")
print(f"Expected (by symmetry): (0.25, 0.25, 0.25)")

Tetrahedron with vertices (0,0,0), (1,0,0), (0,1,0), (0,0,1)
  Computed volume: 0.1666666667
  Expected (1/6): 0.1666666667
  Error: 1.11e-14

Center of mass: (0.250000, 0.250000, 0.250000)
Expected (by symmetry): (0.25, 0.25, 0.25)


## N-Dimensional Integration

For arbitrary dimensions:

**Function**: `scipy.integrate.nquad(func, ranges)`

**Parameters**:
- `func(*args)`: Function taking n arguments
- `ranges`: List of integration ranges [(a1,b1), (a2,b2), ...]

Each range can be:
- Tuple `(a, b)` for fixed limits
- Tuple `(gfun, hfun)` for variable limits

**More flexible** than dblquad/tplquad

In [10]:
# 2D example using nquad: ∫₀¹ ∫₀¹ xy dx dy = 1/4
def f_2d(x, y):
    return x * y

result, error = integrate.nquad(
    f_2d,
    [(0, 1), (0, 1)]  # x from 0 to 1, y from 0 to 1
)

print("2D with nquad: ∫₀¹ ∫₀¹ xy dy dx")
print(f"  Result: {result:.10f}")
print(f"  Expected: 0.25")

# 3D example: ∫₀¹ ∫₀¹ ∫₀¹ xyz dz dy dx = 1/8
def f_3d(x, y, z):
    return x * y * z

result, error = integrate.nquad(
    f_3d,
    [(0, 1), (0, 1), (0, 1)]
)

print("\n3D with nquad: ∫₀¹ ∫₀¹ ∫₀¹ xyz dz dy dx")
print(f"  Result: {result:.10f}")
print(f"  Expected: {1/8:.10f}")

# 4D example: ∫₀¹ ∫₀¹ ∫₀¹ ∫₀¹ xyzw dw dz dy dx = 1/16
def f_4d(x, y, z, w):
    return x * y * z * w

result, error = integrate.nquad(
    f_4d,
    [(0, 1), (0, 1), (0, 1), (0, 1)]
)

print("\n4D with nquad: ∫₀¹ ∫₀¹ ∫₀¹ ∫₀¹ xyzw dw dz dy dx")
print(f"  Result: {result:.10f}")
print(f"  Expected: {1/16:.10f}")
print(f"\nnquad works for any dimension!")

2D with nquad: ∫₀¹ ∫₀¹ xy dy dx
  Result: 0.2500000000
  Expected: 0.25

3D with nquad: ∫₀¹ ∫₀¹ ∫₀¹ xyz dz dy dx
  Result: 0.1250000000
  Expected: 0.1250000000

4D with nquad: ∫₀¹ ∫₀¹ ∫₀¹ ∫₀¹ xyzw dw dz dy dx
  Result: 0.0625000000
  Expected: 0.0625000000

nquad works for any dimension!


## nquad with Variable Limits

For variable limits, provide functions:

```python
ranges = [
    (a, b),                    # x: fixed
    (lambda x: g(x), lambda x: h(x)),  # y: depends on x
    (lambda x, y: p(x,y), lambda x, y: q(x,y))  # z: depends on x,y
]
```

**Order**: Innermost integral first (reverse of mathematical notation)

In [11]:
# Triangle using nquad: 0 ≤ x ≤ 1, 0 ≤ y ≤ x
result, error = integrate.nquad(
    lambda y, x: 1.0,
    [(lambda x: 0, lambda x: x),  # y from 0 to x (inner)
     (0, 1)]                       # x from 0 to 1 (outer)
)

print("Triangle area using nquad:")
print(f"  Result: {result:.10f}")
print(f"  Expected: 0.5")

# Cone: 0 ≤ x ≤ 1, 0 ≤ y ≤ x, 0 ≤ z ≤ y
# Volume = ∫₀¹ ∫₀ˣ ∫₀ʸ dz dy dx
result, error = integrate.nquad(
    lambda z, y, x: 1.0,
    [(lambda x, y: 0, lambda x, y: y),  # z from 0 to y
     (lambda x: 0, lambda x: x),         # y from 0 to x
     (0, 1)]                             # x from 0 to 1
)

print("\nCone-like volume (0≤z≤y≤x≤1):")
print(f"  Result: {result:.10f}")
# Analytical: ∫₀¹ ∫₀ˣ y dy dx = ∫₀¹ x²/2 dx = x³/6|₀¹ = 1/6
print(f"  Expected (1/6): {1/6:.10f}")

TypeError: '<' not supported between instances of 'function' and 'function'

## Physics: Moment of Inertia

Moment of inertia about z-axis for 3D solid:

\[ I_z = \iiint (x^2 + y^2) \rho(x,y,z) \, dV \]

Where ρ is density.

**Example**: Solid cylinder (radius R, height H, uniform density ρ₀)

In cylindrical coordinates: I_z = (1/2)MR²

We'll compute in Cartesian coordinates.

In [12]:
# Cylinder: x² + y² ≤ R², 0 ≤ z ≤ H
R = 1.0  # radius
H = 2.0  # height
rho = 1.0  # density

# Mass = πR²H·ρ
mass, _ = integrate.tplquad(
    lambda z, y, x: rho,
    -R, R,
    lambda x: -np.sqrt(R**2 - x**2),
    lambda x: np.sqrt(R**2 - x**2),
    lambda x, y: 0,
    lambda x, y: H
)

print(f"Cylinder (R={R}, H={H}, ρ={rho})")
print(f"  Computed mass: {mass:.6f}")
print(f"  Expected (πR²Hρ): {np.pi*R**2*H*rho:.6f}")

# Moment of inertia about z-axis: ∫∫∫ (x²+y²)ρ dV
def inertia_integrand(z, y, x):
    return (x**2 + y**2) * rho

I_z, _ = integrate.tplquad(
    inertia_integrand,
    -R, R,
    lambda x: -np.sqrt(R**2 - x**2),
    lambda x: np.sqrt(R**2 - x**2),
    lambda x, y: 0,
    lambda x, y: H
)

print(f"\nMoment of inertia I_z:")
print(f"  Computed: {I_z:.6f}")
print(f"  Expected (MR²/2): {mass*R**2/2:.6f}")
print(f"  Relative error: {abs(I_z - mass*R**2/2)/(mass*R**2/2)*100:.4f}%")

Cylinder (R=1.0, H=2.0, ρ=1.0)
  Computed mass: 6.283185
  Expected (πR²Hρ): 6.283185

Moment of inertia I_z:
  Computed: 3.141593
  Expected (MR²/2): 3.141593
  Relative error: 0.0000%


## Function Comparison

| Function | Dimensions | Argument Order | Flexibility |
|----------|------------|----------------|-------------|
| `dblquad()` | 2D | func(y, x) | Medium |
| `tplquad()` | 3D | func(z, y, x) | Medium |
| `nquad()` | Any | func(x₁, x₂, ...) | High |

### When to Use:

| Use | Reason |
|-----|--------|
| `dblquad()` | Standard 2D integrals, familiar notation |
| `tplquad()` | Standard 3D integrals |
| `nquad()` | Complex variable limits, >3D, flexible order |

### Tips:
- **Remember argument order**: y,x for dblquad; z,y,x for tplquad
- **Variable limits**: Functions can depend on outer variables
- **Performance**: Can be slow for complex regions
- **Alternative**: Monte Carlo for high dimensions

## Monte Carlo Integration (Bonus)

For high-dimensional integrals, Monte Carlo can be faster:

\[ \int_V f(x) \, dx \approx V \cdot \frac{1}{N} \sum_{i=1}^N f(x_i) \]

Where xᵢ are random points in volume V.

**Advantages**:
- Convergence O(1/√N) regardless of dimension
- Simple to implement
- Works for complex regions

**Disadvantages**:
- Slower convergence than deterministic methods for low dimensions
- Requires many samples for accuracy

In [13]:
# Monte Carlo integration: Volume of sphere
def monte_carlo_sphere_volume(r, n_samples=100000):
    """Estimate sphere volume using Monte Carlo"""
    # Generate random points in cube [-r, r]³
    points = np.random.uniform(-r, r, size=(n_samples, 3))
    
    # Check which points are inside sphere
    distances = np.sqrt(np.sum(points**2, axis=1))
    inside = distances <= r
    
    # Volume = (volume of cube) × (fraction inside)
    cube_volume = (2*r)**3
    sphere_volume = cube_volume * np.mean(inside)
    
    return sphere_volume

r = 1.0
expected = 4*np.pi*r**3/3

print(f"Monte Carlo estimation of sphere volume (r={r}):\n")

for n in [1000, 10000, 100000, 1000000]:
    estimate = monte_carlo_sphere_volume(r, n)
    error = abs(estimate - expected)
    rel_error = error/expected * 100
    
    print(f"  N={n:7d}: {estimate:.6f}  (error: {rel_error:.3f}%)")

print(f"\nExpected: {expected:.6f}")
print(f"\nConvergence: O(1/√N) → needs 100x samples for 10x accuracy")

Monte Carlo estimation of sphere volume (r=1.0):

  N=   1000: 4.096000  (error: 2.215%)
  N=  10000: 4.246400  (error: 1.375%)
  N= 100000: 4.180720  (error: 0.193%)
  N=1000000: 4.183408  (error: 0.128%)

Expected: 4.188790

Convergence: O(1/√N) → needs 100x samples for 10x accuracy


## Summary: Key Takeaways

### Integration Functions:

✓ **dblquad(func, a, b, gfun, hfun)**: Double integration, func(y, x)  
✓ **tplquad(func, a, b, gfun, hfun, qfun, rfun)**: Triple integration, func(z, y, x)  
✓ **nquad(func, ranges)**: N-dimensional, flexible argument order  

### Argument Order:
- **dblquad**: func(y, x) - inner variable first
- **tplquad**: func(z, y, x) - innermost first
- **nquad**: func(x₁, x₂, ...) - natural order

### Variable Limits:
- Functions can depend on outer integration variables
- Example: y from 0 to x → `lambda x: 0`, `lambda x: x`
- Example: z from 0 to 1-x-y → `lambda x, y: 1-x-y`

### Applications Covered:
1. **Geometry**: Area, volume of regions
2. **Physics**: Center of mass, moment of inertia
3. **Probability**: Joint distributions, marginal probabilities
4. **Engineering**: Mass, stress, heat distribution

### Performance Tips:
- Simplify regions when possible (symmetry)
- Use appropriate coordinate systems (cylindrical, spherical)
- Monte Carlo for high dimensions (>5)
- Check convergence with error estimates

### Next Steps:
- Learn coordinate transformations (cylindrical, spherical)
- Study ODE solvers (scipy.integrate.solve_ivp)
- Explore PDE solvers for partial differential equations

## Practice Problems

1. **Ellipse area**: Integrate 1 over region x²/a² + y²/b² ≤ 1 (should get πab)

2. **Paraboloid volume**: Volume under z = 1 - x² - y² above xy-plane

3. **Joint probability**: For f(x,y) = 6xy² on 0≤x≤1, 0≤y≤1, find P(X<0.5, Y<0.5)

4. **Pyramid volume**: Vertices (±1,±1,0) and (0,0,1)

5. **Expected distance**: E[√(X²+Y²)] where X,Y ~ Uniform[0,1]

In [14]:
print("PRACTICE SOLUTIONS\n" + "="*60)

# 1. Ellipse area
a, b = 2.0, 3.0
area, _ = integrate.dblquad(
    lambda y, x: 1.0,
    -a, a,
    lambda x: -b*np.sqrt(1 - x**2/a**2),
    lambda x: b*np.sqrt(1 - x**2/a**2)
)
print(f"1. Ellipse area (a={a}, b={b}): {area:.6f}")
print(f"   Expected (πab): {np.pi*a*b:.6f}\n")

# 2. Paraboloid volume
volume, _ = integrate.dblquad(
    lambda y, x: 1 - x**2 - y**2 if x**2 + y**2 <= 1 else 0,
    -1, 1,
    lambda x: -np.sqrt(1 - x**2),
    lambda x: np.sqrt(1 - x**2)
)
print(f"2. Paraboloid volume: {volume:.6f}")
print(f"   Expected (π/2): {np.pi/2:.6f}\n")

# 3. Joint probability
prob, _ = integrate.dblquad(
    lambda y, x: 6*x*y**2,
    0, 0.5,
    lambda x: 0,
    lambda x: 0.5
)
print(f"3. P(X<0.5, Y<0.5): {prob:.6f}\n")

# 4. Pyramid volume
volume, _ = integrate.tplquad(
    lambda z, y, x: 1.0,
    -1, 1,
    lambda x: -1, lambda x: 1,
    lambda x, y: 0, lambda x, y: 1 - max(abs(x), abs(y))
)
print(f"4. Pyramid volume: {volume:.6f}")
print(f"   Expected (base×height/3 = 4×1/3): {4/3:.6f}\n")

# 5. Expected distance
exp_dist, _ = integrate.dblquad(
    lambda y, x: np.sqrt(x**2 + y**2),
    0, 1,
    lambda x: 0,
    lambda x: 1
)
print(f"5. E[√(X²+Y²)]: {exp_dist:.6f}")

print("\n" + "="*60 + "\n✓ All problems solved!")

PRACTICE SOLUTIONS
1. Ellipse area (a=2.0, b=3.0): 18.849556
   Expected (πab): 18.849556

2. Paraboloid volume: 1.570796
   Expected (π/2): 1.570796

3. P(X<0.5, Y<0.5): 0.031250

4. Pyramid volume: 1.333333
   Expected (base×height/3 = 4×1/3): 1.333333

5. E[√(X²+Y²)]: 0.765196

✓ All problems solved!
